In [ ]:
import pandas as pd

import datareader
from problemsolver import ProblemSolver
import solutions

In [ ]:
FILE_PREFERENCES = "voorkeuren_tortels_2024.xlsx"
GROUPS_TO = ["Blauw", "Groen", "Oranje", "Geel"]

In [ ]:
processor = datareader.VoorkeurenProcessor(FILE_PREFERENCES)
preferences = processor.process(all_to_groups=GROUPS_TO)
students_info = processor.get_students_meta_info()
students_per_group_from = processor.get_students_per_old_group()

In [ ]:
ps = ProblemSolver(
    preferences,
    students_per_group_from,
    students_info,
    GROUPS_TO,
    max_imbalance_boys_girls=1,
)



prob = ps.run()

In [ ]:
sa = solutions.SolutionAnalyzer(prob, preferences, processor.input)

display(sa.display_groepsindeling())
display(sa.solution_performance)
display(sa.display_student_performance())
display(sa.display_satisfied_preferences())
sa.to_excel()

# Analysis

In [ ]:
solution_performance_overview = dict()
known_hashes = set()
relevant_solutions = []
optimization_options = [
    "studentsatisfaction",
    "n_preferences",
    "weighted_preferences",
    "least_satisfied",
]
for optimize in optimization_options:
    for n_students_max in range(4, 7):
        for max_imbalance in range(1, 4):
            ps = ProblemSolver(
                preferences,
                students_per_group_from,
                students_info,
                GROUPS_TO,
                optimize=optimize,
                max_clique=n_students_max,
                max_imbalance_boys_girls=max_imbalance,
            )
            prob = ps.run()
            sa = solutions.SolutionAnalyzer(prob, preferences, processor.input)
            new_hash = sa.get_hash()
            if new_hash not in known_hashes:
                known_hashes.add(new_hash)
                relevant_solutions.append(sa)
                print(f"{optimize=} {n_students_max=} {max_imbalance=}")
                display(sa.display_groepsindeling())

                solution_performance_overview[(optimize, n_students_max)] = (
                    sa.solution_performance
                )

In [ ]:
pd.DataFrame.from_dict(solution_performance_overview, orient="index")

In [ ]:
for sol in relevant_solutions:
    display(sol.display_groepsindeling())
    display(sol.solution_performance)
    display(sol.display_student_performance())
    print("-" * 30)